In [22]:
import tensorflow as tf
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from skimage import util
from math import exp, log
from tensorflow.python.keras.engine.base_layer import Layer
from matplotlib import rc
rc('mathtext', default='regular')

In [23]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [24]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()

        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):

        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]
    
    def get_batch_test(self):
    
        return self.test_data, self.test_label
    
    def get_local_noise(self, noise_num=400, batch_size=10000):
        """
        Generate noise_num # of gassaion noise knowing location 
        
        """

        process_image = np.copy(self.test_data[:batch_size])
        W_x = intitial_W_x(process_image)
        uncer_mass = np.zeros(W_x[:,0,0,:,0].shape) + 0.01
        base_rate = np.zeros(W_x[:,0,0,:,0].shape) + 0.5
        image_size = process_image.shape[1]
        index_noise = np.random.randint(0, image_size*image_size, noise_num)
        
        for i in index_noise:
            noise_value = np.random.randn(batch_size,1) * 0.4
            feature_value = process_image[:,int(i/image_size),int(i%image_size),:]
            process_image[:,int(i/image_size),int(i%image_size),:] = feature_value + noise_value
            belief_mass = W_x[:,int(i/image_size),int(i%image_size),:,0] * (1-noise_value)
            disbelief_mass = 1 - belief_mass - uncer_mass
            W_x[:,int(i/image_size),int(i%image_size),:] = np.moveaxis(np.array([belief_mass,disbelief_mass,uncer_mass,base_rate]), 0, -1)
            
        return tf.clip_by_value(process_image, 0, 1), self.test_label[:batch_size], W_x, index_noise

    
    
    def get_noise(self, noise_mode):
        process_image = np.copy(self.test_data)
        noise_gs_img = util.random_noise(process_image,mode=noise_mode,var=0.2)
        return noise_gs_img, self.test_label
    
data_loader = MNISTLoader()

In [25]:
def intitial_opinion_trust(X):
    """
    X - 50,32,32,3
    return W_x - 50,32,32,3,4
    
    """
    W_dis = np.zeros(X.shape).astype(np.float32)
    W_base = W_dis + 0.5
    W_x = np.array([X, W_dis, 1 - X, W_base]).astype(np.float32)
    x_trust = (W_x[0] + W_x[2] * 0.5)
    W_x = np.moveaxis(W_x, 0, -1)
    
    return W_x, x_trust

In [26]:
def intitial_W_x_location(X, index):
    """
    X - 5000,28,28,1
    return W_x - 400,
    
    """
    W_x = intitial_W_x(X)
    uncer_mass = np.zeros(W_x[:,0,0,:,0].shape) + 0.01
    base_rate = np.zeros(W_x[:,0,0,:,0].shape) + 0.5
    image_size = X.shape[1]
    for i in index:
        belief_mass = W_x[:,int(i/image_size),int(i%image_size),:,0]
        disbelief_mass = 1 - belief_mass - uncer_mass
        W_x[:,int(i/image_size),int(i%image_size),:] = np.moveaxis(np.array([belief_mass,disbelief_mass,uncer_mass,base_rate]), 0, -1)

    return W_x

In [27]:
def padding(opinion, kernel_size):
    """
    opinion - (batch_size 500, image_size 32， image_size 32，channel 3, 4) 
    W_w - (kernel_size, kernel_size, 3, filter_num, 4)
    p - int(( n_W_prev - f + 2 * pad )/ stride) + 1
    
    """
    image_size = opinion.shape[1]
    p = int((kernel_size - 1)/2)
    padding_size = int(image_size + 2 * p)
    opinion_pad = np.zeros((int(opinion.shape[0]),padding_size,padding_size,int(opinion.shape[-2]),int(opinion.shape[-1])))
    opinion_pad[:,:,:,:] = np.array([0.0, 0.99, 0.01, 0.5])
    opinion_pad[:,p:p+image_size,p:p+image_size,:,:] = opinion
    return opinion_pad

In [28]:
def multi(W_x, W_y): # 
    W_x = W_x.astype(np.float64) 
    W_y = W_y.astype(np.float64) 
#     print(np.isnan(np.min(W_x[0]*W_y[0])))    
    W_b = W_x[0]*W_y[0]+((1-W_x[3])*W_y[3]*W_x[0]*W_y[2]+W_x[3]*(1-W_y[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
#     print(np.isnan(np.min(W_b)))
    W_d = W_x[1]+W_y[1]-W_x[1]*W_y[1]
    W_u = W_x[2]*W_y[2]+((1-W_y[3])*W_x[0]*W_y[2]+(1-W_x[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_a = W_x[3]*W_y[3]
    return W_b,W_d,W_u,W_a

In [29]:
def conv_single_step_trust(W_x, W_w, W_b): 
    """
    W_x - (50,5,5,3,4)
    W_w - (5,5,3,32,4)
    W_b - (32,4)
    W_wx - (4, 32, 3, 5, 5)
    fusion_result - (32,)
    
    """    
    W_x = W_x.astype(np.float64)   
    W_w = W_w.numpy().astype(np.float64) 
    W_b = W_b.numpy().astype(np.float64) 
    
    filter_number = W_b.shape[0]
    batch_size = W_x.shape[0]
    fusion_result = []
    W_x_expand = np.tile(np.expand_dims(W_x, axis=(4)), [1,1,1,1,filter_number,1])
    W_w_expand = np.tile(np.expand_dims(W_w, axis=(0)), [batch_size,1,1,1,1,1])
    W_b_expand = np.tile(np.expand_dims(W_b, axis=(0)), [batch_size,1,1])
#     print(W_x_expand,W_w_expand)
#     print('W_x_expand',np.isnan(np.min(W_x_expand)))
#     print('W_w_expand',np.isnan(np.min(W_w_expand)))
    W_wx = multi(np.transpose(W_x_expand),np.transpose(W_w_expand)) # (4, 32, 3, 5, 5, 50)
#     print(np.asarray(W_wx)[:])
#     print(np.isnan(np.min(W_wx)))
    fusion_result = avg_fusion(np.asarray(W_wx), np.transpose(W_b_expand))
#     print(np.isnan(np.min(fusion_result[0])))
    return np.transpose(fusion_result[0]),np.transpose(fusion_result[1])

def avg_fusion(W_wx, W_b):
    """
    W_wx - (4, 32, 3, 5, 5, 50)
    W_b -  (4, 32, 50)
    fusion_result - (32, 32, 32)
    
    return opinion - 4,32,50
           trust - 32,50
    """
    
    W_wx = np.reshape(W_wx, (W_wx.shape[0],W_wx.shape[1], 
                             W_wx.shape[2]*W_wx.shape[3]*W_wx.shape[4], W_wx.shape[5])).astype(np.float64) # (4, 32, 75, 50)
    W_b = W_b.astype(np.float64)
    
    n_filter = W_b.shape[1]
    batch_size = W_wx.shape[-1]
    num_para = W_wx.shape[2]
#     print(num_para)
    b_wx, u_wx, a_wx = W_wx[0], W_wx[2], W_wx[3]
    b_b, u_b, a_b = W_b[0], W_b[2], W_b[3]
        
    
    u_combine = np.concatenate((u_wx, np.reshape(u_b,(u_b.shape[0], 1, batch_size))), axis=1) # (32, 76, 50)
    b_combine = np.concatenate((b_wx, np.reshape(b_b,(b_b.shape[0], 1, batch_size))), axis=1) # (32, 76, 50)
    u_combine_recip = (np.zeros(u_combine.shape)+1)/u_combine
    
    numerator = np.sum(b_combine * u_combine_recip, axis = 1) # (32, 50)
    denominator = np.sum(u_combine_recip, axis = (1))
    
    b_fusion = numerator / denominator
    u_fusion = (num_para+1) / denominator
    a_fusion = (np.sum(a_wx, axis=(1)) + a_b) / (num_para+1)
    
    return np.array([b_fusion, 1-b_fusion-u_fusion, u_fusion, a_fusion]).astype(np.float32), (b_fusion + u_fusion * a_fusion).astype(np.float32)

In [30]:
def conv_forward(A_prev, W_w, W_b):
    """
        A_prev - (batch_size 500, image_size 32， image_size 32，channel 3, 4) 
        W_w - (kernel_size, kernel_size, 3, filter_num, 4)
        W_b - (filter_num, 4)
        return Z - (28, 28, filter_num, 4)

    """
    
    #获取来自上一层数据的基本信息
#     print(A_prev.shape,W_w.shape)
    (batch_size, n_H_prev, n_W_prev, _, _) = A_prev.shape
    
    #获取权重矩阵的基本信息
    ( f , f , _, n_C, _) = W_w.shape


    #获取超参数hparameters的值
    stride = 1
    pad = 1
    
    # padding opinion
    A_prev_pad = padding(A_prev,f)
    
    #计算卷积后的图像的宽度高度，参考上面的公式，使用int()来进行板除
    n_H = int(( n_H_prev - f + 2 * pad )/ stride) + 1
    n_W = int(( n_W_prev - f + 2 * pad )/ stride) + 1
 
    
    #使用0来初始化卷积输出Z
    Z = np.zeros((batch_size, n_H, n_W, n_C, 4)) 
    Z_trust = np.zeros((batch_size, n_H, n_W, n_C))

    for h in range(n_H):                       
        for w in range(n_W):                              
            vert_start = h * stride        
            vert_end = vert_start + f       
            horiz_start = w * stride        
            horiz_end = horiz_start + f     
            a_slice_prev = A_prev_pad[:,vert_start:vert_end,horiz_start:horiz_end,:,:]   # 500, 3, 3, 3, 4
#             print(a_slice_prev)
#             print('a_slice_prev :', np.isnan(np.min(a_slice_prev)))
            opinion, trust = conv_single_step_trust(a_slice_prev,W_w,W_b)   
#             print(opinion.shape)
            Z[:,h,w,:,:] = opinion
            Z_trust[:,h,w,:] = trust
#     print(Z.shape,Z_trust.shape)
    return Z, Z_trust

In [31]:
class MyLayerMaxTrust(tf.keras.layers.Layer):
    """
    max-trust fuction layer
    
    """
    def __init__(self):
        super(MyLayerMaxTrust, self).__init__()


    def call(self, x, opinion, trust):
        """
        x - (50, 28, 28, 32)
        opinion - (50, 28, 28, 32, 4)
        trust - (50, 28, 28, 32)
        
        x, pooling_opinion1 = self.maxtrust(x, opinion1, trust1)

        opinion_out - (50, 14, 14, 32, 4)
        return - (50, 14, 14, 32, 4)

        """
    #         print(inputs.shape,inputB.shape)
        trust_mul = np.zeros(x.shape)
        image_size = x.shape[1]
        opinion_out = np.zeros((opinion.shape[0],int(opinion.shape[1]/2), int(opinion.shape[2]/2),opinion.shape[3], 4)).astype(np.float32)
#         print(opinion_out.shape)
        for i in range(trust.shape[0]):
            for k in range(trust.shape[-1]):
                input_max = trust[i,:,:,k].reshape((1,trust.shape[1],trust.shape[2],1))
                _, argmax = tf.nn.max_pool_with_argmax(input = input_max, ksize = [1, 2, 2, 1],
                                                    strides = [1, 2, 2, 1], padding = 'VALID')
                argmax_1d = argmax.numpy().flatten()
#                 print(len(argmax_1d))
            
                for j in range(len(argmax_1d)):
                    trust_mul[i,int(argmax_1d[j]/image_size),int(argmax_1d[j]%image_size),k] = 1
                    opinion_out[i,int(j/int(image_size/2)), int(j%int(image_size/2)),k,:] = opinion[i,int(argmax_1d[j]/image_size),int(argmax_1d[j]%image_size),k,:] 

        x = x * trust_mul

        return x, opinion_out

In [32]:
class MyLayerCONV(tf.keras.layers.Layer):
    """
    max-trust fuction layer
    
    """
    def __init__(self):
        super(MyLayerCONV, self).__init__()


    def call(self, input1, input2, input3):
        
        Z, Z_trust = conv_forward(input1, input2, input3)
           
        return Z.astype(np.float32), Z_trust.astype(np.float32)

In [33]:
class MyLayerMaxPool(tf.keras.layers.Layer):
    """
    original max-pooling with trust
     
    """
    def __init__(self):
        super(MyLayerMaxPool, self).__init__()


    def call(self, x, opinion):
        """
        x-(50,14,14,8)
        
        """
        image_size = x.shape[1]
        opinion_out = np.zeros((opinion.shape[0],int(opinion.shape[1]/2), 
                                int(opinion.shape[2]/2),opinion.shape[3], 4)).astype(np.float32)

        for i in range(x.shape[0]):
            for k in range(x.shape[-1]):
                input_max = x[i,:,:,k].numpy().reshape((1,x.shape[1],x.shape[2],1))
                _, argmax = tf.nn.max_pool_with_argmax(input = input_max, ksize = [1, 2, 2, 1],
                                                    strides = [1, 2, 2, 1], padding = 'VALID')
                argmax_1d = argmax.numpy().flatten()
                for j in range(len(argmax_1d)):
                    opinion_out[i,int(j/int(image_size/2)), int(j%int(image_size/2)),k,:] = opinion[i,int(argmax_1d[j]/image_size),
                                                                                                    int(argmax_1d[j]%image_size),k,:] 
        return opinion_out

In [34]:
class MyLayerComputeTrust(tf.keras.layers.Layer):
    """
    input- (50,14,14,8)
    
    """
    def __init__(self):
        super(MyLayerComputeTrust, self).__init__()


    def call(self, input1):
        
        opinion, trust = intitial_opinion_trust(input1)
           
        return opinion, trust

In [35]:
def get_update_matrix(grads_w, grads_b, rs):
    """
    grads_list_w1 = grads_list_w + tf.where(abs(tf.squeeze(grads[0]))<0.01, 1, 0)  #  shape=(5, 5, 3, 32)
    grads_list_b1 = grads_list_b + tf.where(abs(tf.squeeze(grads[1]))<0.01, 1, 0)  #  shape=(32,)
    grads_list_w2 = grads_list_w + tf.where(abs(tf.squeeze(grads[0]))<0.01, 1, 0)  #  shape=(5, 5, 32, 64)
    grads_list_b2 = grads_list_b + tf.where(abs(tf.squeeze(grads[1]))<0.01, 1, 0)  #  shape=(64,)
    grads_list_w3 = grads_list_w + tf.where(abs(tf.squeeze(grads[0]))<0.01, 1, 0)  #  shape=(5, 5, 64, 64)
    grads_list_b3 = grads_list_b + tf.where(abs(tf.squeeze(grads[1]))<0.01, 1, 0)  #  shape=(64,)
    
    """
    
    W_w = np.array([grads_w/(rs+2),(rs-grads_w)/(rs+2),np.full(grads_w.shape, 2/(rs+2)),
                    np.full(grads_w.shape, 0.5)]).astype(np.float32)
    W_b = np.array([grads_b/(rs+2), (rs-grads_b)/(rs+2), np.full(grads_b.shape, 2/(rs+2)), 
                    np.full(grads_b.shape, 0.5)]).astype(np.float32)
    W_w = np.moveaxis(W_w, 0, -1)
    
    return W_w, W_b.swapaxes(0,1)

In [36]:
filter_num1 = 4
filter_num2 = 8
filter_num3 = 8
kernelsize = 3

In [37]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.convopinion1 = MyLayerCONV()
        self.convopinion2 = MyLayerCONV()
        self.convopinion3 = MyLayerCONV()
        self.compute_trust = MyLayerComputeTrust()

        self.conv1 = tf.keras.layers.Conv2D(
            filters=filter_num1,             
            kernel_size=[kernelsize, kernelsize],
            padding='same',
            activation=tf.nn.relu   
        )
        self.conv2 = tf.keras.layers.Conv2D(
            filters=filter_num2,           
            kernel_size=[kernelsize, kernelsize],    
            padding='same',
            activation=tf.nn.relu   
        )
        self.conv3 = tf.keras.layers.Conv2D(
            filters=filter_num3,            
            kernel_size=[kernelsize, kernelsize],     
            padding='same',         
            activation=tf.nn.relu   
        )

        self.maxtrust1 = MyLayerMaxTrust()
        self.maxtrust2 = MyLayerMaxTrust()
        self.maxfeature1 = MyLayerMaxPool()
        self.maxfeature2 = MyLayerMaxPool()
        
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
       
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)

    def call(self, inputs):
        """
        x - 50,32,32,3
        W_x - 50,32,32,3,4
        
        
        """
        X, W_x, W_w1, W_b1, W_w2, W_b2 = inputs
 
        opinion1, trust1 = self.convopinion1(W_x, W_w1, W_b1)
        x = self.conv1(X)   
#         print(np.isnan(np.min(opinion1)))
#         print('Underflow or not: ',np.isnan(np.min(trust1)))
#         print(opinion1.shape,trust1.shape)
#         x, pooling_opinion1 = self.maxtrust1(x, opinion1, trust1)
        pooling_opinion1 = self.maxfeature1(x, opinion1)
        x = self.pool1(x) 
#         print('First 2 layers end:',(time.time()-start))
#         print(np.isnan(np.min(x)))
#         print(np.isnan(np.min(pooling_opinion1)))
        opinion2, _ = self.convopinion2(pooling_opinion1, W_w2, W_b2)
        x = self.conv2(x)
        
        pooling_opinion2 = self.maxfeature2(x, opinion2)
        x = self.pool2(x)
#         print(np.isnan(np.min(opinion2)))
#         print(np.isnan(np.min(x)))
#         print('Second 2 layers end:',(time.time()-start))
        
        x = self.flatten(x) 
        x = self.dense1(x)    
        output = self.dense2(x)                    
        return output,opinion1,opinion2,pooling_opinion2
    
model = CNN()

In [38]:
num_epochs = 2
batch_size = 50
threshold1 = 3.0
threshold2 = 5.0
optimizer = tf.keras.optimizers.Adam()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [39]:
# initial opinion
# W_x = np.array([[[[0.0, 0.0, 1.0, 0.5] for _ in range(3)] for _ in range(32)] for _ in range(32)]).astype(np.float32)

w_initial1 = np.array([[[[[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num1)] 
                         for _ in range(1)] for _ in range(kernelsize)] for _ in range(kernelsize)]).astype(np.float32)
b_initial1 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num1)]).astype(np.float32)

w_initial2 = np.array([[[[[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num2)] 
                         for _ in range(filter_num1)] for _ in range(kernelsize)] for _ in range(kernelsize)]).astype(np.float32)
b_initial2 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num2)]).astype(np.float32)

# w_initial3 = np.array([[[[[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num3)] 
#                          for _ in range(filter_num2)] for _ in range(kernelsize)] for _ in range(kernelsize)]).astype(np.float32)
# b_initial3 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(filter_num3)]).astype(np.float32)

# w_initial4 = np.array([[[0.0, 0.0, 1.0, 0.5] for _ in range(64)] for _ in range(784)]).astype(np.float32)
# b_initial4 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(64)]).astype(np.float32)

# w_initial5 = np.array([[[0.0, 0.0, 1.0, 0.5] for _ in range(10)] for _ in range(64)]).astype(np.float32)
# b_initial5 = np.array([[0.0, 0.0, 1.0, 0.5] for _ in range(10)]).astype(np.float32)

In [40]:
def evidence_collect(y, y_pred):
    r = 0
    s = 0
    r_list = [0]*10
    s_list = [0]*10
    
    for j in range(len(y_pred)):
        for i in range(len(y_pred[0])):
            if i == y[j]:
                if y_pred[j][i] > 0.8:
                    r_list[i]+=1
                    r+=1
                else:
                    s+=1
                    s_list[i]+=1
            else:
                if y_pred[j][i] < 0.2:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
                    
    y_N_op = []
    for i in range(len(r_list)):
        y_N_op.append([r_list[i]/(r_list[i]+s_list[i]+2), 
                       s_list[i]/(r_list[i]+s_list[i]+2), 2/(r_list[i]+s_list[i]+2), 0.5])

    
    return [r/(r+s+2), s/(r+s+2), 2/(r+s+2), 0.5], y_N_op

In [41]:
train_acc = []
train_loss = []
opinion_convlist = []
y_update_wb = []
y_N_opinion = []

In [42]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)

W_w1 = w_initial1
W_b1 = b_initial1
W_w2 = w_initial2
W_b2 = b_initial2
# W_w3 = w_initial3
# W_b3 = b_initial3
# W_w4 = w_initial4
# W_b4 = b_initial4
# W_w5 = w_initial5
# W_b5 = b_initial5

grads_list_w1 = np.zeros((kernelsize,kernelsize,1,filter_num1))
grads_list_b1 = np.zeros((filter_num1,))
grads_list_w2 = np.zeros((kernelsize,kernelsize,filter_num1,filter_num2))
grads_list_b2 = np.zeros((filter_num2,))
# grads_list_w3 = np.zeros((kernelsize,kernelsize,filter_num2,filter_num3))
# grads_list_b3 = np.zeros((filter_num3,))
# grads_list_w4 = np.zeros((784,64))
# grads_list_b4 = np.zeros((64,))
# grads_list_w5 = np.zeros((64,10))
# grads_list_b5 = np.zeros((10,))

import time
start = time.time()

for batch_index in range(num_batches):
    print('# of batch:',batch_index)
    X, y = data_loader.get_batch(batch_size)  
    W_x ,_ = intitial_opinion_trust(X)
    with tf.GradientTape() as tape:
        y_pred,opinion_conv1,opinion_conv2,opinion_conv = model([X, W_x, W_w1, W_b1, W_w2, W_b2])
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    
    sparse_categorical_accuracy.update_state(y_true=y, y_pred=y_pred)
    opinion_convlist.append(np.reshape(opinion_conv, (opinion_conv.shape[0], 
                                                      int(opinion_conv.shape[1]*opinion_conv.shape[2]*opinion_conv.shape[3]),4)))
    train_loss.append(tf.reduce_mean(loss).numpy())
    train_acc.append(sparse_categorical_accuracy.result().numpy())
    print("train accuracy:",sparse_categorical_accuracy.result().numpy())
#     print('Model update end:',(time.time()-start))
    
    
# update W_w, W_b
    grads_list_w1 = grads_list_w1 + tf.where(abs(grads[0])<threshold1, 1, 0)  #  shape=(5, 5, 3, 8)
    grads_list_b1 = grads_list_b1 + tf.where(abs(grads[1])<threshold1, 1, 0)  #  shape=(8,)
    grads_list_w2 = grads_list_w2 + tf.where(abs(grads[2])<threshold2, 1, 0)  #  shape=(5, 5, 8, 12)
    grads_list_b2 = grads_list_b2 + tf.where(abs(grads[3])<threshold2, 1, 0)  #  shape=(12,)
#     grads_list_w3 = grads_list_w3 + tf.where(abs(grads[4])<threshold, 1, 0)  #  shape=(5, 5, 12, 12)
#     grads_list_b3 = grads_list_b3 + tf.where(abs(grads[5])<threshold, 1, 0)  #  shape=(12,)
#     grads_list_w4 = grads_list_w4 + tf.where(abs(grads[6])<threshold, 1, 0)  #  shape=(5, 5, 12, 12)
#     grads_list_b4 = grads_list_b4 + tf.where(abs(grads[7])<threshold, 1, 0)  #  shape=(12,)
#     grads_list_w5 = grads_list_w5 + tf.where(abs(grads[8])<threshold, 1, 0)  #  shape=(5, 5, 12, 12)
#     grads_list_b5 = grads_list_b5 + tf.where(abs(grads[9])<threshold, 1, 0)  #  shape=(12,)
    
#     print('W_w, W_b update start:',(time.time()-start))  # 20s
    
    W_w1, W_b1 = get_update_matrix(grads_list_w1, grads_list_b1, batch_index+1)
    W_w2, W_b2 = get_update_matrix(grads_list_w2, grads_list_b2, batch_index+1)
#     W_w3, W_b3 = get_update_matrix(grads_list_w3, grads_list_b3, batch_index+1)
#     W_w4, W_b4 = get_update_matrix(grads_list_w4, grads_list_b4, batch_index+1)
#     W_w5, W_b5 = get_update_matrix(grads_list_w5, grads_list_b5, batch_index+1)

    y_N_update, y_N_op = evidence_collect(y, y_pred)
    
    y_N_opinion.append(y_N_op)
    y_update_wb.append(y_N_update)

    print('One batch end:',(time.time()-start))

# of batch: 0
train accuracy: 0.06
One batch end: 2.780397653579712
# of batch: 1
train accuracy: 0.05
One batch end: 3.5443973541259766
# of batch: 2
train accuracy: 0.1
One batch end: 4.306440830230713
# of batch: 3
train accuracy: 0.12
One batch end: 5.072985649108887
# of batch: 4
train accuracy: 0.116
One batch end: 5.8425068855285645
# of batch: 5
train accuracy: 0.13
One batch end: 6.641559839248657
# of batch: 6
train accuracy: 0.13428572
One batch end: 7.6015589237213135
# of batch: 7
train accuracy: 0.1475
One batch end: 8.372765302658081
# of batch: 8
train accuracy: 0.15777777
One batch end: 9.31676459312439
# of batch: 9
train accuracy: 0.184
One batch end: 10.269825220108032
# of batch: 10
train accuracy: 0.18727273
One batch end: 11.164836883544922
# of batch: 11
train accuracy: 0.20166667
One batch end: 11.940836429595947
# of batch: 12
train accuracy: 0.22153845
One batch end: 12.72896146774292
# of batch: 13
train accuracy: 0.22285715
One batch end: 13.499016284942627

train accuracy: 0.6398214
One batch end: 86.78405046463013
# of batch: 112
train accuracy: 0.6421239
One batch end: 87.50957608222961
# of batch: 113
train accuracy: 0.6433333
One batch end: 88.36857533454895
# of batch: 114
train accuracy: 0.6450435
One batch end: 89.20257496833801
# of batch: 115
train accuracy: 0.64689654
One batch end: 89.95157551765442
# of batch: 116
train accuracy: 0.64871794
One batch end: 90.68857026100159
# of batch: 117
train accuracy: 0.65118647
One batch end: 91.4152626991272
# of batch: 118
train accuracy: 0.65260506
One batch end: 92.19126200675964
# of batch: 119
train accuracy: 0.6545
One batch end: 92.98376822471619
# of batch: 120
train accuracy: 0.65504134
One batch end: 93.7617678642273
# of batch: 121
train accuracy: 0.65737706
One batch end: 94.55876731872559
# of batch: 122
train accuracy: 0.6593496
One batch end: 95.34976696968079
# of batch: 123
train accuracy: 0.6608065
One batch end: 96.13276648521423
# of batch: 124
train accuracy: 0.66304


train accuracy: 0.759
One batch end: 172.4312608242035
# of batch: 220
train accuracy: 0.7591855
One batch end: 173.14133262634277
# of batch: 221
train accuracy: 0.7598198
One batch end: 173.86585426330566
# of batch: 222
train accuracy: 0.7606278
One batch end: 174.62885355949402
# of batch: 223
train accuracy: 0.7608036
One batch end: 175.34985303878784
# of batch: 224
train accuracy: 0.76115555
One batch end: 176.05985260009766
# of batch: 225
train accuracy: 0.76203537
One batch end: 176.78485226631165
# of batch: 226
train accuracy: 0.76255506
One batch end: 177.4998517036438
# of batch: 227
train accuracy: 0.76324564
One batch end: 178.2028512954712
# of batch: 228
train accuracy: 0.7638428
One batch end: 178.90585064888
# of batch: 229
train accuracy: 0.7646087
One batch end: 179.60785055160522
# of batch: 230
train accuracy: 0.76545453
One batch end: 180.31984996795654
# of batch: 231
train accuracy: 0.76586205
One batch end: 181.02584958076477
# of batch: 232
train accuracy: 

train accuracy: 0.810061
One batch end: 257.2130515575409
# of batch: 328
train accuracy: 0.8103343
One batch end: 257.9575881958008
# of batch: 329
train accuracy: 0.81078786
One batch end: 258.67319774627686
# of batch: 330
train accuracy: 0.8108157
One batch end: 259.3908817768097
# of batch: 331
train accuracy: 0.8111446
One batch end: 260.25196528434753
# of batch: 332
train accuracy: 0.81147146
One batch end: 260.9584903717041
# of batch: 333
train accuracy: 0.8117365
One batch end: 261.6684923171997
# of batch: 334
train accuracy: 0.8118806
One batch end: 262.3744852542877
# of batch: 335
train accuracy: 0.8120238
One batch end: 263.1860806941986
# of batch: 336
train accuracy: 0.81216615
One batch end: 264.04660081863403
# of batch: 337
train accuracy: 0.8123077
One batch end: 264.79059624671936
# of batch: 338
train accuracy: 0.81262535
One batch end: 265.51060009002686
# of batch: 339
train accuracy: 0.813
One batch end: 266.23759961128235
# of batch: 340
train accuracy: 0.81

train accuracy: 0.83647597
One batch end: 339.95542454719543
# of batch: 437
train accuracy: 0.83684933
One batch end: 340.6624240875244
# of batch: 438
train accuracy: 0.8370843
One batch end: 341.3724238872528
# of batch: 439
train accuracy: 0.8373182
One batch end: 342.0834228992462
# of batch: 440
train accuracy: 0.83746034
One batch end: 342.79142236709595
# of batch: 441
train accuracy: 0.8377828
One batch end: 343.5069432258606
# of batch: 442
train accuracy: 0.8379232
One batch end: 344.23894357681274
# of batch: 443
train accuracy: 0.8381081
One batch end: 344.9509425163269
# of batch: 444
train accuracy: 0.83833706
One batch end: 345.6635401248932
# of batch: 445
train accuracy: 0.8386099
One batch end: 346.3685395717621
# of batch: 446
train accuracy: 0.8387472
One batch end: 347.09053897857666
# of batch: 447
train accuracy: 0.8389286
One batch end: 347.8022356033325
# of batch: 448
train accuracy: 0.8391091
One batch end: 348.51023530960083
# of batch: 449
train accuracy: 

...

train accuracy: 0.92932236
One batch end: 1576.7374675273895
# of batch: 1948
train accuracy: 0.92935866
One batch end: 1577.688717365265
# of batch: 1949
train accuracy: 0.9293846
One batch end: 1578.618236064911
# of batch: 1950
train accuracy: 0.9294106
One batch end: 1579.5372359752655
# of batch: 1951
train accuracy: 0.9294365
One batch end: 1580.4687569141388
# of batch: 1952
train accuracy: 0.9294521
One batch end: 1581.3997569084167
# of batch: 1953
train accuracy: 0.92946774
One batch end: 1582.3387529850006
# of batch: 1954
train accuracy: 0.9294629
One batch end: 1583.258862733841
# of batch: 1955
train accuracy: 0.9294888
One batch end: 1584.18186211586
# of batch: 1956
train accuracy: 0.9295146
One batch end: 1585.087861776352
# of batch: 1957
train accuracy: 0.92953014
One batch end: 1585.9708609580994
# of batch: 1958
train accuracy: 0.9295559
One batch end: 1586.8808591365814
# of batch: 1959
train accuracy: 0.92958164
One batch end: 1587.7988584041595
# of batch: 1960


train accuracy: 0.93141603
One batch end: 1674.8665833473206
# of batch: 2055
train accuracy: 0.93143
One batch end: 1675.7931027412415
# of batch: 2056
train accuracy: 0.93144387
One batch end: 1676.6801056861877
# of batch: 2057
train accuracy: 0.9314675
One batch end: 1677.6201045513153
# of batch: 2058
train accuracy: 0.9314813
One batch end: 1678.4951045513153
# of batch: 2059
train accuracy: 0.93150485
One batch end: 1679.4131035804749
# of batch: 2060
train accuracy: 0.9315187
One batch end: 1680.3340997695923
# of batch: 2061
train accuracy: 0.9315422
One batch end: 1681.2551023960114
# of batch: 2062
train accuracy: 0.9315657
One batch end: 1682.1636154651642
# of batch: 2063
train accuracy: 0.9315795
One batch end: 1683.0796165466309
# of batch: 2064
train accuracy: 0.9316126
One batch end: 1683.974615097046
# of batch: 2065
train accuracy: 0.93161666
One batch end: 1684.8792080879211
# of batch: 2066
train accuracy: 0.9316304
One batch end: 1685.774207353592
# of batch: 2067

train accuracy: 0.93340427
One batch end: 1772.2148010730743
# of batch: 2162
train accuracy: 0.93341655
One batch end: 1773.160322189331
# of batch: 2163
train accuracy: 0.9334196
One batch end: 1774.1343214511871
# of batch: 2164
train accuracy: 0.9334226
One batch end: 1775.0809836387634
# of batch: 2165
train accuracy: 0.93344414
One batch end: 1776.0009138584137
# of batch: 2166
train accuracy: 0.9334656
One batch end: 1776.9269132614136
# of batch: 2167
train accuracy: 0.9334779
One batch end: 1777.8574242591858
# of batch: 2168
train accuracy: 0.93346244
One batch end: 1778.9580302238464
# of batch: 2169
train accuracy: 0.9334654
One batch end: 1780.047027349472
# of batch: 2170
train accuracy: 0.93349606
One batch end: 1781.1470596790314
# of batch: 2171
train accuracy: 0.9335175
One batch end: 1782.1540586948395
# of batch: 2172
train accuracy: 0.9335389
One batch end: 1783.179579257965
# of batch: 2173
train accuracy: 0.93356943
One batch end: 1784.2215900421143
# of batch: 2

train accuracy: 0.93522257
One batch end: 1875.6214821338654
# of batch: 2269
train accuracy: 0.9352511
One batch end: 1876.6599988937378
# of batch: 2270
train accuracy: 0.9352796
One batch end: 1877.6480021476746
# of batch: 2271
train accuracy: 0.9352993
One batch end: 1878.6155169010162
# of batch: 2272
train accuracy: 0.93531895
One batch end: 1879.597029209137
# of batch: 2273
train accuracy: 0.9353298
One batch end: 1880.5635368824005
# of batch: 2274
train accuracy: 0.9353319
One batch end: 1881.5070431232452
# of batch: 2275
train accuracy: 0.9353515
One batch end: 1882.4610407352448
# of batch: 2276
train accuracy: 0.9353711
One batch end: 1883.4140439033508
# of batch: 2277
train accuracy: 0.9353556
One batch end: 1884.4080431461334
# of batch: 2278
train accuracy: 0.9353664
One batch end: 1885.4370384216309
# of batch: 2279
train accuracy: 0.93539476
One batch end: 1886.5145978927612
# of batch: 2280
train accuracy: 0.9354143
One batch end: 1887.5275931358337
# of batch: 22

One batch end: 1978.2697739601135
# of batch: 2375
train accuracy: 0.9369613
One batch end: 1979.388299703598
# of batch: 2376
train accuracy: 0.9369794
One batch end: 1980.381294965744
# of batch: 2377
train accuracy: 0.9370059
One batch end: 1981.318818807602
# of batch: 2378
train accuracy: 0.9370071
One batch end: 1982.3783173561096
# of batch: 2379
train accuracy: 0.9370168
One batch end: 1983.4601881504059
# of batch: 2380
train accuracy: 0.93703485
One batch end: 1984.5271909236908
# of batch: 2381
train accuracy: 0.9370445
One batch end: 1985.513712644577
# of batch: 2382
train accuracy: 0.9370625
One batch end: 1986.3943111896515
# of batch: 2383
train accuracy: 0.93706375
One batch end: 1987.4233105182648
# of batch: 2384
train accuracy: 0.937065
One batch end: 1988.4138135910034
# of batch: 2385
train accuracy: 0.93709135
One batch end: 1989.484820842743
# of batch: 2386
train accuracy: 0.9371177
One batch end: 1990.4728157520294
# of batch: 2387
train accuracy: 0.93711895
O

In [43]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
 
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

In [47]:
model.save_weights('model_max_pool_MINIST1_sub')
save_variable(W_w1, 'max_pool_MINST1_weight_opinion1_sub')
save_variable(W_w2, 'max_pool_MINST1_weight_opinion2_sub')

save_variable(W_b1, 'max_pool_MINIST1_bias_opinion1_sub')
save_variable(W_b2, 'max_pool_MINIST1_bias_opinion2_sub')

'max_pool_MINIST1_bias_opinion2_sub)'

In [45]:
# model.load_weights('my_model_max_trust_MINIST')
# W_w1 = load_variavle('MINST_weight_opinion1')
# W_w2 = load_variavle('MINST_weight_opinion2')
# W_b1 = load_variavle('MINIST_bias_opinion1')
# W_b2 = load_variavle('MINIST_bias_opinion2')

In [52]:
X_test, y_test= data_loader.get_batch_test()
W_x ,_ = intitial_opinion_trust(X_test)
y_pred_test, _,_,_ = model([X_test, W_x, W_w1, W_b1, W_w2, W_b2])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=y_test, y_pred=y_pred_test)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

test accuracy: 0.9719


In [53]:
# test noisy dataset
X_test_noise, y_test_noise = data_loader.get_noise('gaussian')
noise_opinion,_ = intitial_opinion_trust(X_test_noise)
y_pred_test_noise,opinion_conv1,opinion_conv2,opinion_conv = model([X_test_noise, noise_opinion, W_w1, W_b1, W_w2, W_b2])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=y_test_noise, y_pred=y_pred_test_noise)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

test accuracy: 0.6267


In [67]:
# test localvar
X_test_local_noise, y_test_local_noise, case1_opinion, noise_index = data_loader.get_local_noise()
case2_opinion = intitial_W_x_location(X_test_local_noise, noise_index)
case3_opinion = intitial_W_x(X_test_local_noise)
y_pred_test_local_noise,_,_,opinion_noise = model([X_test_local_noise, case1_opinion, W_w1, W_b1, W_w2, W_b2])
sparse_categorical_accuracy.reset_states()
sparse_categorical_accuracy.update_state(y_true=y_test_local_noise, y_pred=y_pred_test_local_noise)
print("test accuracy:",sparse_categorical_accuracy.result().numpy())

test accuracy: 0.8536


In [58]:
tf.nn.moments(tf.constant([0.7659,0.5648,0.573,0.9616,0.9515]),axes=0)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.76336>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.030073041>)

In [ ]:
def get_NN_trust(opinion_last_layer, true_label, pre_label, y_update_wb):
    # compute dense opinion
    opinion_dense = np.average(np.array(opinion_last_layer), axis=0)
    opinion_dense = np.reshape(opinion_dense, 
                                     (int(opinion_dense.shape[0]*opinion_dense.shape[1]*opinion_dense.shape[2]),4))
    y_true_op = [1.0, 0.0, 0.0, 0.5]
    W_y_update = evidence_collect_test(true_label, pre_label)
    
    # compute Backward opinion of neuron W_N_Y  
    W_N_Y=[]
    for j in W_y_update:
        W_N_Y.append(multi(j,y_true_op)) # change when add flaw in label
        
    W_w_dense = y_update_wb[-1]
    W_b_dense = y_update_wb[-1]

    W_xw=[]
    W_xw.append(W_b_dense)
    for j in range(opinion_dense.shape[0]):
        W_xw.append(multi(W_w_dense,opinion_dense[j])) # (513, 4)
    dense_out = fusion(np.array(W_xw))
    #     print('Underflow or not: ',np.isnan(np.min(np.array(dense1_out_list))))

    # last layer
    W_xw=[]
    W_xw.append(W_b_dense)
    for j in range(64):
        W_xw.append(multi(W_w_dense,dense_out))
    W_NN = fusion(np.array(W_xw))
    
    # compute last layer output opinion and trust
    W_XY_one = []
    for j in range(10):
        W_XY_one.append(fusion_2(W_NN,W_N_Y[j]))
    W_NN = fusion(np.array(W_XY_one))
    W_trust = W_NN[0]+W_NN[2]*W_NN[3]
    
    return W_trust, W_NN

In [ ]:
# forward opinion prop
def mul_scale(W_x):
    mul_u = 1.0
    for i in range(len(W_x)):
        mul_u = mul_u * W_x[i]
        if mul_u <= 1e-24: 
            mul_u = mul_u*1e25
    return mul_u

def fusion(W_x): # W_x array

    deno = 0.0
    mole = 0.0
    full_multi = mul_scale(W_x[:,2])
    for i in range(len(W_x[:,2])):
        deno = deno + full_multi/W_x[:,2][i]
        mole = mole + (full_multi/W_x[:,2][i])*W_x[:,0][i]
    W_b = mole/deno  #  change
    W_u = (len(W_x)*full_multi)/deno   # change
    W_a = sum(W_x[:,3])/len(W_x)  

#     W_b = sum(1/len(W_x)*W_x[:,0])
#     W_u = 0.0
#     W_a = sum(1/len(W_x)*W_x[:,3])
        
    return [W_b,1-W_b-W_u,W_u,W_a]

def fusion_2(W_x, W_y):
    if W_x[2]!=0 or W_y[2]!=0:
        W_b = (W_x[0]*W_y[2]+W_y[0]*W_x[2])/(W_x[2]+W_y[2])
        W_u = 2*W_x[2]*W_y[2]/(W_x[2]+W_y[2])
        W_a = (W_x[3]+W_y[3])/2  
    elif W_x[2]==0 and W_y[2]==0:
        W_b = 0.5*W_x[0]+0.5*W_y[0]
        W_u = 0
        W_a = 0.5*W_x[3]+0.5*W_y[3]
    return [W_b,1-W_b-W_u,W_u,W_a]

def multi(W_x, W_y): # 
    W_b = W_x[0]*W_y[0]+((1-W_x[3])*W_y[3]*W_x[0]*W_y[2]+W_x[3]*(1-W_y[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_d = W_x[1]+W_y[1]-W_x[1]*W_y[1]
    W_u = W_x[2]*W_y[2]+((1-W_y[3])*W_x[0]*W_y[2]+(1-W_x[3])*W_y[0]*W_x[2])/(1-W_x[3]*W_y[3])
    W_a = W_x[3]*W_y[3]
    return [W_b,W_d,W_u,W_a]


def evidence_collect_test(y, y_pred):
    r_list = [0]*10
    s_list = [0]*10
    index = np.random.randint(0,y.shape[0], 5000)
    
    for j in index:
        for i in range(len(y_pred[0])):
            if i == y[j]:
                if y_pred[j][i] > 0.9:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
            else:
                if y_pred[j][i] < 0.1:
                    r_list[i]+=1
                else:
                    s_list[i]+=1
                    
    y_N_op = []
    for i in range(len(r_list)):
        y_N_op.append(np.array([r_list[i]/(r_list[i]+s_list[i]+2), 
                       s_list[i]/(r_list[i]+s_list[i]+2), 2/(r_list[i]+s_list[i]+2), 0.5]))

    
    return y_N_op

# def evidence_collect_test(y, y_pred):
#     r_list = 0
#     s_list = 0
#     index = np.random.randint(0,y.shape[0], 1000)
    
#     for i in index:
#         if np.argmax(y_pred[i]) == y[i]:
#             r_list+=1
#         else:
#             s_list+=1
                    
#     return np.array([r_list/(r_list+s_list+2), s_list/(r_list+s_list+2), 2/(r_list+s_list+2), 0.5])

In [ ]:
print('Original image trustworthiness result:',get_NN_trust(opinion_original, y_test, y_pred_test, y_update_wb)[1])

In [ ]:
print('Noise image trustworthiness result:',get_NN_trust(opinion_noise, y_test_noise, y_pred_test_noise, y_update_wb)[1])

In [ ]:
W_y_update = evidence_collect_test(y_test_noise, y_pred_test_noise)

In [ ]:
# compute Backward opinion of neuron W_N_Y  
W_N_Y_noise=[]
for j in W_y_update:
    W_N_Y_noise.append(multi(j,y_true_op)) # change when add flaw in label

In [ ]:
opinion_dense_noise = np.average(np.array(opinion_mid), axis=0)
opinion_dense_noise = np.reshape(opinion_dense_noise, 
                                 (int(opinion_dense_noise.shape[0]*opinion_dense_noise.shape[1]*opinion_dense_noise.shape[2]),4))

In [ ]:
W_w_dense = y_update_wb[-1]
W_b_dense = y_update_wb[-1]
input_dense = opinion_dense_noise

W_xw=[]
W_xw.append(W_b_dense)
for j in range(opinion_dense_noise.shape[0]):
    W_xw.append(multi(W_w_dense,input_dense[j]))
dense_out_noise = fusion(np.array(W_xw))
#     print('Underflow or not: ',np.isnan(np.min(np.array(dense1_out_list))))

# last layer
W_xw=[]
W_xw.append(W_b_dense)
for j in range(64):
    W_xw.append(multi(W_w_dense,dense_out_noise))
W_NN_noise = fusion(np.array(W_xw))

In [ ]:
# compute last layer output opinion and trust
W_XY_one = []
for j in range(10):
    W_XY_one.append(fusion_2(W_NN_noise,W_N_Y_noise[j]))

W_NN = fusion(np.array(W_XY_one))
W_trust = W_NN[0]+W_NN[2]*W_NN[3]

In [ ]:
W_trust

In [ ]:
entropy = -1 * (W_trust * np.log2(W_trust) + (1-W_trust) * np.log2(1-W_trust))
entropy

In [ ]:
plt.imshow(X_test[0])
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
for i in range(1,11):
    plt.subplot(1,10,i)
    plt.imshow(trust1[0,:,:,i-1])
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
for i in range(1,21):
    plt.subplot(1,20,i)
    plt.imshow(trust2[0,:,:,i-1])
plt.show()

In [ ]:
trust2[0,1,1,0]

In [ ]:
plt.figure()
for i in range(1,13):
    plt.subplot(1,12,i)
    plt.imshow(trust3[0,:,:,i-1])
plt.show()

In [ ]:
opinion1 = np.moveaxis(opinion1, -1, 0)
opinion2 = np.moveaxis(opinion2, -1, 0)

In [ ]:
opinion1.shape

In [ ]:
max_trust1 = opinion1[0]+opinion1[2]*opinion1[3]
max_trust2 = opinion2[0]+opinion2[2]*opinion2[3]

In [ ]:
max_trust1.shape

In [ ]:
plt.figure()
for i in range(1,7):
    plt.subplot(1,6,i)
    plt.imshow(max_trust1[0,:,:,i-1])
plt.show()

In [ ]:
plt.figure()
for i in range(1,13):
    plt.subplot(1,12,i)
    plt.imshow(max_trust2[0,:,:,i-1])
plt.show()